In [ ]:
#Import required packages
import os
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
import sklearn
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score
from tensorflow.keras import optimizers
from sklearn.metrics import roc_curve, auc, precision_recall_curve,average_precision_score, confusion_matrix, f1_score,matthews_corrcoef
from inspect import signature

os.environ["HOME"]

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
#Define the function to make one-hot encoded matrix of peptide sequence
def one_hot_encoding(s):
    pep_num = ' '.join([str(ints) for ints in range(20)])
    pep_ref = 'A C D E F G H I K L M N P Q R S T V W Y'
    d = dict(zip(pep_ref.split(' '), pep_num.split(' ')))
    
    x = np.zeros((len(d), len(s)))
    x[[int(d[c]) for c in s], range(len(s))] = 1
    return x

In [ ]:
#folder_input = 'INPUT_TABLE_FILE_PATH'
#To train the model, we used input data from in vivo with peptide sequence and class information

m_input = pd.read_csv('{}/Input4000_ML_m.csv'.format(folder), sep = "\t", index_col = 0)

In [ ]:
#Separate the train and test dataset as the ratio of 9:1
m_train = invivo_4000.groupby('Class', group_keys = False).apply(sampling_func, sample_pct = 0.9)
m_test = invivo_4000.drop(invivo_4000.index[m_train.index])
pep_train = m_train['Pep_seq'].tolist()
pep_test = m_test['Pep_seq'].tolist()

In [ ]:
#Make the train and test input from peptide sequences
X_train = [one_hot_encoding(x) for x in pep_train]
X_train = np.transpose(np.asarray(X_train), (0, 2, 1))
X_test = [one_hot_encoding(x) for x in pep_test]
X_test = np.transpose(np.asarray(X_test), (0, 2, 1))
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

Y_train = np.array(m_train['Class'].tolist())
Y_test = np.array(m_test['Class'].tolist())

print('Length of training data:', X_train.shape[0])
print('Length of peptides:', X_train.shape[1])
print('Length of amino acids:', X_train.shape[2])

In [ ]:
#Specify the model path
model_path = "PATH_TO_CROSS-VALIDATED_MODEL"
model_path = os.path.join(model_path,'MODEL_NAME')
os.makedirs(model_path, exist_ok = True)
print(model_path)

In [ ]:
#Hyperparameter setting

BATCH_SIZE = 8
PEP_length = 12
AA_type  = 20
CLASS_NAMES = ['enriched','not-enriched']
params = [['CONV', 400, 3, 1],
          ['DROP', 0.5],
          ['POOL', 2, 1],
          ['FLAT'],
          ['DENSE', 50]]
activation_func = 'relu'
regularizer_params = None
steps_per_epoch = 450
Epochs = 50

In [ ]:
model_version = 1
dict_history = {}

#Randomly split the data as the 9:1 ratio for 10-fold cross validation
split_data = ShuffleSplit(n_splits = 10, train_size = None, test_size = 0.1, random_state = 1)

acc_per_fold = []
loss_per_fold = []
val_acc_per_fold = []
val_loss_per_fold = []
roc_per_fold = []
roc_fpr = []
roc_tpr = []
precision_fold = []
recall_fold = []
ave_precision_fold = []
fold_num = []
n_iter =0

for train_idx, test_idx in split_data.split(X_seq, Y_class):
    X_train = np.array(X_seq[train_idx])
    X_test = np.array(X_seq[test_idx])
    y_train = np.array(Y_class[train_idx])
    y_test = np.array(Y_class[test_idx])
    #Define the model structure
    model = models.Sequential()
    model.add(layers.Conv1D(filters = params[0][1],kernel_size = params[0][2],strides = params[0][3],activation = activation_func, input_shape = (PEP_length,AA_type), kernel_regularizer = regularizer_params, bias_regularizer = regularizer_params, padding = 'same'))
    model.add(layers.Dropout(rate = params[1][1]))
    model.add(layers.MaxPool1D(pool_size = params[2][1], strides = params[2][2]))
    model.add(layers.Flatten())
    model.add(layers.Dense(params[4][1], activation = activation_func, kernel_regularizer = regularizer_params,bias_regularizer = regularizer_params))
    model.add(layers.Dense(1, activation = 'sigmoid'))
    #Compile the model
    model.compile(loss='binary_crossentropy', optimizer = optimizers.Adam(learning_rate = 0.000075), metrics = ['acc'] )
    dict_history[model_version] = model.fit(x = X_train,y = y_train,shuffle = True, steps_per_epoch = steps_per_epoch, epochs = Epochs, batch_size = BATCH_SIZE, validation_data = (X_test, y_test), verbose = 2)
    history = dict_history[model_version]
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    scores = cross_validate(model, X_test, y_test, verbose = 0)
    print(f'Score for fold {n_iter + 1}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%\n')
    acc_per_fold.append(acc)
    loss_per_fold.append(loss)
    val_acc_per_fold.append(val_acc)
    val_loss_per_fold.append(val_loss)
    arr_acc = np.array(acc_per_fold)
    arr_loss = np.array(loss_per_fold)
    arr_val_acc = np.array(val_acc_per_fold)
    arr_val_loss = np.array(val_loss_per_fold)
    epochs = range(1, len(acc) + 1)
    pred = model.predict(x = X_test)
    fpr, tpr, thresholds = roc_curve(y_test, pred)
    roc_fpr.append(fpr)
    roc_tpr.append(tpr)
    roc_auc = auc(fpr, tpr)
    roc_per_fold.append(roc_auc)
    precision, recall, thresholds = precision_recall_curve(y_test, pred)
    average_precision = average_precision_score(y_test, pred)
    precision_fold.append(precision)
    recall_fold.append(recall)
    ave_precision_fold.append(average_precision)
    globals()['model_fname_{}'.format(n_iter+1)] = f'BBBphagedisplay_TEST_{n_iter+1}_{model_version:03}.h5'
    model_spath = os.path.join(model_path,f'model_fname_{n_iter+1}')
    model.save(model_spath)
    fold_num.append(n_iter + 1)
    n_iter += 1
    epochs = range(1, len(acc) + 1)
    #Get the graph of accuracy of training and validation accuracy (10-fold CV)
    fig = plt.figure(figsize = (10.5,3.5))
    plt.plot(epochs, acc, 'bo', label = 'Training acc')
    plt.plot(epochs, val_acc, 'r', alpha = 0.7, label = 'Validation acc')
    plt.title(f'Training and validation accuracy of fold {n_iter+1}')
    plt.legend()
    #Get the graph of accuracy of training and validation loss (10-fold CV)
    fig = plt.figure(figsize = (10.5,3.5))
    plt.plot(epochs, loss, 'bo', label = 'Training loss')
    plt.plot(epochs, val_loss, 'r', alpha = 0.7, label = 'Validation loss')
    plt.title(f'Training and validation loss of fold {n_iter+1}')
    plt.legend()
    plt.show()
    #Get the graph of an receiver operating characteristic (ROC) curve (10-fold CV)
    pred = model.predict(x = X_test)
    fpr, tpr, thresholds = roc_curve(y_test, pred)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, color = 'darkorange',
             lw = 2, label = 'ROC curve (area - %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color = 'navy', lw = 2, linestyle = '--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(loc = 'lower right')
    plt.show()
    plt.cla()
    plt.clf()
    #Get the graph of a precision-recall curve (10-fold CV)
    precision, recall, thresholds = precision_recall_curve(y_test, pred)
    average_precision = average_precision_score(y_test, pred)
    step_kwargs = ({'step': 'post'}
                   if 'step' in signature(plt.fill_between).parameters
                   else {})
    plt.step(recall, precision, color = 'navy', alpha = 0.2, where = 'post', label = 'Avg. Precision: {0:0.2f}'.format(average_precision))
    plt.fill_between(recall, precision, alpha = 0.2, color = 'navy', **step_kwargs)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.legend(loc = 'lower right')
    plt.title(f'Precision recall curve and ROC curve of fold {n_iter+1}')
    plt.show()
    plt.cla()
    plt.clf()

In [ ]:
#The cross-validated model was saved in the "PATH_TO_CROSS-VALIDATED_MODEL"
globals()['model_fname'] = f'BBBpeptidePredictor_{model_version:03}.h5'
model_spath = os.path.join(model_path, 'model_name')
model.save(model_spath)